In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Video, display



def detect_green_pitch_final_veo(frame):
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    lower_green = np.array([45, 70, 80])
    upper_green = np.array([70, 255, 255])
    green_mask = cv2.inRange(hsv, lower_green, upper_green)

    kernel = np.ones((9, 9), np.uint8)
    green_mask = cv2.morphologyEx(green_mask, cv2.MORPH_OPEN, kernel)
    green_mask = cv2.morphologyEx(green_mask, cv2.MORPH_CLOSE, kernel)

    height, width = green_mask.shape
    region_mask = np.zeros_like(green_mask)
    region_mask[
        int(height * 0.25):int(height * 0.79),
        int(width * 0.08):int(width * 0.92)
    ] = 255
    green_mask = cv2.bitwise_and(green_mask, region_mask)

    contours, _ = cv2.findContours(green_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    pitch_mask = np.zeros_like(green_mask)

    for cnt in contours:
        area = cv2.contourArea(cnt)
        x, y, w, h = cv2.boundingRect(cnt)
        aspect_ratio = w / float(h) if h > 0 else 0
        solidity = area / float(w * h) if w * h > 0 else 0
        center_y = y + h / 2

        if (
            area > width * height * 0.08 and
            1.8 < aspect_ratio < 5 and
            solidity > 0.7 and
            height * 0.3 < center_y < height * 0.9
        ):
            cv2.drawContours(pitch_mask, [cnt], -1, 255, -1)

    return pitch_mask

def draw_grid_on_green_field(frame, green_mask):
    grid_frame = frame.copy()
    height, width = frame.shape[:2]

    field_corners = np.float32([
        [width * 0.10, height * 0.10],   # Top-left corner of field (adjusted)
        [width * 0.90, height * 0.10],   # Top-right corner of field (adjusted)
        [width * 0.99, height * 0.99],   # Bottom-right corner of field (adjusted)
        [width * 0.01, height * 0.99]    # Bottom-left corner of field (adjusted)
    ])
    
    num_cols = 8
    num_rows = 6
    
    grid_points = []
    for row in range(num_rows + 1):
        for col in range(num_cols + 1):
            x_norm = col / num_cols
            y_norm = row / num_rows
            grid_points.append([x_norm, y_norm])
    
    grid_points = np.array(grid_points, dtype=np.float32)
    dst_corners = np.float32([[0, 0], [1, 0], [1, 1], [0, 1]])
    
    # Calculate perspective transformation matrix
    M = cv2.getPerspectiveTransform(dst_corners, field_corners)
    
    # Transform all grid points
    grid_points_homogeneous = np.column_stack([grid_points, np.ones(len(grid_points))])
    transformed_points = M.dot(grid_points_homogeneous.T).T
    transformed_points = transformed_points[:, :2] / transformed_points[:, 2:3]
    
    # Draw horizontal lines
    for row in range(num_rows + 1):
        start_idx = row * (num_cols + 1)
        end_idx = start_idx + num_cols
        
        # Ensure points are within image bounds before drawing
        start_point = tuple(map(int, transformed_points[start_idx]))
        end_point = tuple(map(int, transformed_points[end_idx]))
        
        cv2.line(grid_frame, start_point, end_point, (255, 255, 255), 2)
    
    # Draw vertical lines  
    for col in range(num_cols + 1):
        points_in_col = transformed_points[col::num_cols + 1]
        
        for i in range(len(points_in_col) - 1):
            # Ensure points are within image bounds before drawing
            start_point = tuple(map(int, points_in_col[i]))
            end_point = tuple(map(int, points_in_col[i + 1]))
            
            cv2.line(grid_frame, start_point, end_point, (255, 255, 255), 2)
    
    # Apply grid only to green field areas
    green_mask_colored = cv2.merge([green_mask] * 3)
    final_frame = np.where(green_mask_colored == 255, grid_frame, frame)
    
    return final_frame

video_path  = r"C:\Users\PC\Desktop\Football_Project\Pressure_metric\test_video\4sec - Made with Clipchamp.mp4"
output_path = r"C:\Users\PC\Desktop\Football_Project\Pressure_metric\test_video\veo_pitch_grid_output.mp4"

cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("❌ Error: Cannot open video.")
else:
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    out_size = (960, 540)

    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    out = cv2.VideoWriter(output_path, fourcc, fps, out_size)

    frame_idx = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.resize(frame, out_size)
        green_mask = detect_green_pitch_final_veo(frame)
        grid_overlay = draw_grid_on_green_field(frame, green_mask)
        out.write(grid_overlay)
        frame_idx += 1

        if frame_idx % 50 == 0:
            print(f"Processed {frame_idx} frames...")

    cap.release()
    out.release()
    print("✅ Done. Output saved as:", output_path)

Processed 50 frames...
Processed 100 frames...
✅ Done. Output saved as: C:\Users\PC\Desktop\Football_Project\Pressure_metric\test_video\veo_pitch_grid_output.mp4
